# 02 - Feature Engineering for Credit Risk Analytics

**Objective:** Create derived features to improve credit risk prediction models

**Key Features to Create:**
- Credit utilization ratio
- Debt-to-income buckets
- FICO score trends
- Loan-to-income ratio
- Risk indicators
- Time-based features

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Libraries imported successfully")

## 1. Load Data

In [ ]:
# Load sample data
df = pd.read_csv('../data/sample/sample_loans_10k.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Check data types and missing values
print("Data Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0])

## 2. Feature Engineering

### 2.1 Credit Utilization Ratio

In [ ]:
# Credit utilization = revolving balance / total credit limit
# Higher utilization = higher risk

if 'revol_bal' in df.columns and 'revol_util' in df.columns:
    # If revol_util exists, use it; otherwise calculate
    df['credit_utilization'] = df['revol_util']
elif 'revol_bal' in df.columns and 'total_acc' in df.columns:
    # Estimate total credit limit
    df['credit_utilization'] = (df['revol_bal'] / (df['revol_bal'] + 1000)) * 100
    df['credit_utilization'] = df['credit_utilization'].clip(0, 100)

# Create utilization buckets
df['util_bucket'] = pd.cut(df['credit_utilization'], 
                           bins=[0, 30, 50, 75, 100],
                           labels=['Low (0-30%)', 'Medium (30-50%)', 'High (50-75%)', 'Very High (75-100%)'])

print("Credit Utilization Distribution:")
print(df['util_bucket'].value_counts())

# Visualize
plt.figure(figsize=(10, 5))
df['credit_utilization'].hist(bins=50, edgecolor='black')
plt.xlabel('Credit Utilization (%)')
plt.ylabel('Frequency')
plt.title('Distribution of Credit Utilization')
plt.axvline(df['credit_utilization'].median(), color='red', linestyle='--', label=f'Median: {df["credit_utilization"].median():.1f}%')
plt.legend()
plt.show()

### 2.2 Debt-to-Income (DTI) Buckets

In [ ]:
# DTI ratio buckets - industry standard thresholds
if 'dti' in df.columns:
    df['dti_bucket'] = pd.cut(df['dti'], 
                              bins=[0, 10, 20, 30, 100],
                              labels=['Low (<10%)', 'Medium (10-20%)', 'High (20-30%)', 'Very High (>30%)'])
    
    print("DTI Bucket Distribution:")
    print(df['dti_bucket'].value_counts())
    
    # Visualize
    plt.figure(figsize=(10, 5))
    df['dti'].hist(bins=50, edgecolor='black')
    plt.xlabel('Debt-to-Income Ratio (%)')
    plt.ylabel('Frequency')
    plt.title('Distribution of DTI Ratio')
    plt.axvline(df['dti'].median(), color='red', linestyle='--', label=f'Median: {df["dti"].median():.1f}%')
    plt.legend()
    plt.show()

### 2.3 FICO Score Features

In [ ]:
# FICO score range and trend
if 'fico_range_low' in df.columns and 'fico_range_high' in df.columns:
    # Average FICO score
    df['fico_score'] = (df['fico_range_low'] + df['fico_range_high']) / 2
    
    # FICO categories (standard credit score ranges)
    df['fico_category'] = pd.cut(df['fico_score'],
                                  bins=[0, 580, 670, 740, 800, 850],
                                  labels=['Poor (<580)', 'Fair (580-670)', 'Good (670-740)', 
                                         'Very Good (740-800)', 'Excellent (800+)'])
    
    print("FICO Category Distribution:")
    print(df['fico_category'].value_counts())
    
    # Visualize
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    df['fico_score'].hist(bins=50, edgecolor='black')
    plt.xlabel('FICO Score')
    plt.ylabel('Frequency')
    plt.title('FICO Score Distribution')
    
    plt.subplot(1, 2, 2)
    df['fico_category'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
    plt.xlabel('FICO Category')
    plt.ylabel('Count')
    plt.title('FICO Category Distribution')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

### 2.4 Loan-to-Income Ratio

In [ ]:
# Loan amount relative to annual income
if 'loan_amnt' in df.columns and 'annual_inc' in df.columns:
    df['loan_to_income'] = (df['loan_amnt'] / df['annual_inc']) * 100
    
    # Cap extreme values
    df['loan_to_income'] = df['loan_to_income'].clip(0, 100)
    
    # Create buckets
    df['lti_bucket'] = pd.cut(df['loan_to_income'],
                              bins=[0, 10, 20, 30, 100],
                              labels=['Low (<10%)', 'Medium (10-20%)', 'High (20-30%)', 'Very High (>30%)'])
    
    print("Loan-to-Income Statistics:")
    print(df['loan_to_income'].describe())
    print("\nLTI Bucket Distribution:")
    print(df['lti_bucket'].value_counts())
    
    # Visualize
    plt.figure(figsize=(10, 5))
    df['loan_to_income'].hist(bins=50, edgecolor='black')
    plt.xlabel('Loan-to-Income Ratio (%)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Loan-to-Income Ratio')
    plt.axvline(df['loan_to_income'].median(), color='red', linestyle='--', 
               label=f'Median: {df["loan_to_income"].median():.1f}%')
    plt.legend()
    plt.show()

### 2.5 Time-Based Features

In [ ]:
# Extract time features if issue_d column exists
if 'issue_d' in df.columns:
    df['issue_date'] = pd.to_datetime(df['issue_d'], format='%b-%Y', errors='coerce')
    
    df['issue_year'] = df['issue_date'].dt.year
    df['issue_month'] = df['issue_date'].dt.month
    df['issue_quarter'] = df['issue_date'].dt.quarter
    
    # Loan vintage (cohort)
    df['vintage'] = df['issue_date'].dt.to_period('M')
    
    print("Loan Issuance by Year:")
    print(df['issue_year'].value_counts().sort_index())
    
    # Visualize
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    df['issue_year'].value_counts().sort_index().plot(kind='bar', color='coral', edgecolor='black')
    plt.xlabel('Year')
    plt.ylabel('Number of Loans')
    plt.title('Loan Origination by Year')
    
    plt.subplot(1, 2, 2)
    df['issue_quarter'].value_counts().sort_index().plot(kind='bar', color='lightgreen', edgecolor='black')
    plt.xlabel('Quarter')
    plt.ylabel('Number of Loans')
    plt.title('Loan Origination by Quarter')
    plt.tight_layout()
    plt.show()

### 2.6 Employment Length Features

In [ ]:
# Convert employment length to numeric
if 'emp_length' in df.columns:
    def parse_emp_length(emp_str):
        if pd.isna(emp_str):
            return np.nan
        if '< 1' in str(emp_str):
            return 0
        if '10+' in str(emp_str):
            return 10
        try:
            return int(str(emp_str).split()[0])
        except:
            return np.nan
    
    df['emp_length_years'] = df['emp_length'].apply(parse_emp_length)
    
    # Employment stability bucket
    df['emp_stability'] = pd.cut(df['emp_length_years'],
                                 bins=[-1, 1, 3, 5, 11],
                                 labels=['New (<1yr)', 'Early (1-3yr)', 'Stable (3-5yr)', 'Established (5+yr)'])
    
    print("Employment Stability Distribution:")
    print(df['emp_stability'].value_counts())

### 2.7 Risk Indicators

In [ ]:
# Create composite risk score (0-100 scale)
# Higher score = higher risk

risk_score = 0

# FICO contribution (40% weight)
if 'fico_score' in df.columns:
    # Normalize FICO (lower FICO = higher risk)
    fico_risk = 100 - ((df['fico_score'] - 300) / (850 - 300) * 100)
    risk_score += fico_risk * 0.4

# DTI contribution (30% weight)
if 'dti' in df.columns:
    # Normalize DTI (higher DTI = higher risk)
    dti_risk = (df['dti'].clip(0, 50) / 50) * 100
    risk_score += dti_risk * 0.3

# Credit utilization contribution (20% weight)
if 'credit_utilization' in df.columns:
    util_risk = df['credit_utilization']
    risk_score += util_risk * 0.2

# Delinquencies contribution (10% weight)
if 'delinq_2yrs' in df.columns:
    delinq_risk = (df['delinq_2yrs'].clip(0, 5) / 5) * 100
    risk_score += delinq_risk * 0.1

df['risk_score'] = risk_score.clip(0, 100)

# Risk categories
df['risk_category'] = pd.cut(df['risk_score'],
                             bins=[0, 25, 50, 75, 100],
                             labels=['Low Risk', 'Medium Risk', 'High Risk', 'Very High Risk'])

print("Risk Score Statistics:")
print(df['risk_score'].describe())
print("\nRisk Category Distribution:")
print(df['risk_category'].value_counts())

# Visualize
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
df['risk_score'].hist(bins=50, edgecolor='black', color='salmon')
plt.xlabel('Risk Score (0-100)')
plt.ylabel('Frequency')
plt.title('Distribution of Risk Scores')

plt.subplot(1, 2, 2)
df['risk_category'].value_counts().plot(kind='bar', color='lightcoral', edgecolor='black')
plt.xlabel('Risk Category')
plt.ylabel('Count')
plt.title('Risk Category Distribution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Feature Summary

In [ ]:
# List all engineered features
engineered_features = [
    'credit_utilization', 'util_bucket',
    'dti_bucket',
    'fico_score', 'fico_category',
    'loan_to_income', 'lti_bucket',
    'issue_year', 'issue_month', 'issue_quarter', 'vintage',
    'emp_length_years', 'emp_stability',
    'risk_score', 'risk_category'
]

existing_features = [f for f in engineered_features if f in df.columns]

print(f"Total engineered features created: {len(existing_features)}")
print(f"\nFeatures: {existing_features}")

# Display sample of engineered features
print("\nSample of engineered features:")
df[existing_features].head(10)

## 4. Feature Correlation Analysis

In [ ]:
# Select numeric features for correlation analysis
numeric_features = df[existing_features].select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_features) > 1:
    # Calculate correlation matrix
    corr_matrix = df[numeric_features].corr()
    
    # Visualize
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated features
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    
    if high_corr:
        print("\nHighly correlated features (|r| > 0.7):")
        for feat1, feat2, corr in high_corr:
            print(f"  {feat1} <-> {feat2}: {corr:.3f}")

## 5. Save Processed Data

In [ ]:
# Save to processed folder
output_path = '../data/processed/loans_with_features.csv'
df.to_csv(output_path, index=False)

print(f"✓ Processed data saved to: {output_path}")
print(f"  Shape: {df.shape}")
print(f"  Total features: {len(df.columns)}")
print(f"  Engineered features: {len(existing_features)}")

## 6. Feature Engineering Summary

### Features Created:

1. **Credit Utilization Metrics**
   - `credit_utilization`: Percentage of credit used
   - `util_bucket`: Categorized utilization levels

2. **Debt Metrics**
   - `dti_bucket`: Categorized debt-to-income ratios
   - `loan_to_income`: Loan amount as % of income
   - `lti_bucket`: Categorized loan-to-income ratios

3. **Credit Score Features**
   - `fico_score`: Average FICO score
   - `fico_category`: Standard credit score categories

4. **Time Features**
   - `issue_year`, `issue_month`, `issue_quarter`: Temporal features
   - `vintage`: Loan cohort identifier

5. **Employment Features**
   - `emp_length_years`: Numeric employment length
   - `emp_stability`: Employment stability categories

6. **Risk Indicators**
   - `risk_score`: Composite risk score (0-100)
   - `risk_category`: Risk level categories

### Next Steps:
- Use these features in credit scoring models (Notebook 03)
- Perform customer segmentation analysis (Notebook 04)
- Build predictive models for default risk